# Desplegar un modelo de Red Neuronal Artificial para consumir en Flask para clasificación de la polaridad del texto
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador3.png" width="900">



## Pantalla del clasificador

<img src="figs/fig_app_web_clasificador.png" width="600">

### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### En el caso de textos corto a nivel de oración, un vector denso por oración. 

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 2 capas, con funciones PReLU en las capas ocultas y una capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   

### 8. **Guardar el modelo**:  
   - Después del entrenamiento, el modelo se guarda para posteriores usos: despliegue del modelo para aplicación web, o como punto de partida para optimizar por medio de Computación Evolutiva.

### 9. **Desplegar el modelo en un entorno para consumir por el usuario**:  
   - Desplegar el modelo entrenado y probado que obtuvo resultados aceptables en la experimentación en términos del rendimiento de las métricas como F1-score, Precisión y Recall.

   Ejemplos:
   - APP WEB
   - API REST
   - Etc.
   
      

# Cargar los datos de entrenamiento

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import fasttext


# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset = pd.read_json("./datasets/dataset_polaridad_es.json", lines=True)

#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())



# Cargar el modelo de Word Embeddings y crear los vectores densos

In [3]:
# Cargar el modelo de word embeddings
ft = fasttext.load_model('/Volumes/data/temp/MX.bin')


# Crear los vectores densos para cada texto. Se espera una oración corta

In [ ]:
# Extracción de los textos en arreglos de numpy
# Se aplica una función lambda para obtener el vector de cada texto del conjunto de datos
# El resultado es una nueva columna "embedding"
dataset["embedding"] = dataset["text"].map(lambda x: ft.get_sentence_vector(x))
dataset.head()

# Crear la matriz de datos para el entrenamiento, validación y prueba del modelo de la red neuronal

In [ ]:
# Cada renglón representa un documento codificado en un texto de embeddings
X = np.vstack(dataset['embedding'].to_numpy())
Y = dataset['klass'].to_numpy()

print("Datos:", X.shape) 
print("Etiquetas:", Y.shape) 



# Codificar las etiquetas

In [ ]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [7]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

# Dividir el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)


# Codificar las clases en forma one-hot 
NUM_CLASSES = 3
Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()
Y_test_one_hot = nn.functional.one_hot(torch.from_numpy(Y_test), num_classes=NUM_CLASSES).float()
Y_val_one_hot = nn.functional.one_hot(torch.from_numpy(Y_val), num_classes=NUM_CLASSES).float()



# Crear minibatches en PyTorch usando DataLoader
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader


In [ ]:
X_train.shape, Y_train.shape, X_val.shape, Y_val.shape, X_test.shape, Y_test.shape

In [ ]:
len(Y_train), len(Y_train_one_hot), Y_train[:5], Y_train_one_hot[:5], 

# Definición de la arquitectura de la red

In [10]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_salida):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        tam_entrada_capa_oculta_1 = 128
        tam_entrada_capa_oculta_2 = 8 
        self.fc1 = nn.Linear(tam_entrada, tam_entrada_capa_oculta_1)
        self.fc2 = nn.Linear(tam_entrada_capa_oculta_1, tam_entrada_capa_oculta_2)
        self.salida = nn.Linear(tam_entrada_capa_oculta_2, tam_salida)
        self.activacion1= nn.PReLU()
        self.activacion2= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.salida.weight)

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.activacion1(x)
        x = self.fc2(x)
        x = self.activacion2(x)
        x = self.salida(x)
        return x

# Entrenamiento de la red

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Establecer los parámetros de la red

# Parámetros de la red
tam_entrada =  X_train.shape[1]

tam_salida = 3   # 3 clases

epochs = 30 # variar el número de épocas, para probar que funciona la programación 
                 # solo usar 2 épocas, para entrenamiento total usar por ejemplo 1000 épocas
learning_rate = 0.001 # Generalmente se usan learning rate pequeños (0.001), 

# Se recomiendan tamaños de batch_size potencias de 2: 16, 32, 64, 128, 256
# Entre mayor el número más cantidad de memoria se requiere para el procesamiento
batch_size = 128 # definir el tamaño del lote de procesamiento 


# Convertir los datos de entrenamiento y etiquetas a tensores  de PyTorch

X_tensor_train = torch.from_numpy(X_train)
X_tensor_train = X_tensor_train.to(torch.float32)

X_tensor_val = torch.from_numpy(X_val)
X_tensor_val = X_tensor_val.to(torch.float32)

X_tensor_test = torch.from_numpy(X_test)
X_tensor_test = X_tensor_train.to(torch.float32)


# Crear la red
modelo_red_neuronal = RedNeuronal(tam_entrada, tam_salida)

# Definir la función de pérdida
# Entropía Cruzada 
criterion = nn.CrossEntropyLoss() 

# Definir el optimizador
# Parámetros del optimizador: parámetros del modelo y learning rate 
# Adaptive Moment Estimation
optimizer = optim.Adam(modelo_red_neuronal.parameters(), lr=learning_rate)

# Entrenamiento
print("Iniciando entrenamiento en PyTorch")

for epoch in range(epochs):
# Poner el modelo en modo de entrenamiento
    modelo_red_neuronal.train()  
    lossTotal = 0
    #definir el batch_size
    dataloader = create_minibatches(X_tensor_train, Y_train_one_hot, batch_size=batch_size)
    for X_tr, y_tr in dataloader:
        # inicializar los gradientes en cero para cada época
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = modelo_red_neuronal(X_tr)  #invoca al método forward de la clase MLP
        # Calcular el error MSE
        loss = criterion(y_pred, y_tr)
        #Acumular el error 
        lossTotal += loss.item()
        
        # Propagación hacia atrás: cálculo de los gradientes de los pesos y bias
        loss.backward()
        
        # actualización de los pesos: regla de actualización basado en el gradiente:
        #  
        optimizer.step()
        if np.random.random() < 0.1:
            print(f"Batch Error : {loss.item()}")

    print(f"Época {epoch+1}/{epochs}, Pérdida: {lossTotal/len(dataloader)}")
    
    # Evalúa el modelo con el conjunto de validación
    modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"
    with torch.no_grad():  # No  calcular gradientes 
        y_pred = modelo_red_neuronal(X_tensor_val)
        # Obtiene una única clase, la más probable
        y_pred = torch.argmax(y_pred, dim=1)
        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_pred, average='macro'))
        print("R=", recall_score(Y_val, y_pred, average='macro'))
        print("F1=", f1_score(Y_val, y_pred, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred))


### Modo para predicción de datos

In [ ]:
# TODO: Transformar el dataset de test con los mismos preprocesamientos y al  espacio de 
# representación vectorial que el modelo entrenado, es decir, al espacio de la matriz TFIDF

# Convertir los datos de prueba a tensores de PyTorch

X_tensor_test = torch.from_numpy(X_test)
X_tensor_test = X_tensor_test.to(torch.float32)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    y_pred_test= modelo_red_neuronal(X_tensor_test)

# y_test_pred contiene las predicciones

# Obtener la clase real
y_pred_test = torch.argmax(y_pred_test, dim=1)

print(f"total predicciones: {len(y_pred_test)}")


### Evaluación

In [ ]:
# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(confusion_matrix(Y_test, y_pred_test))
print(classification_report(Y_test, y_pred_test, digits=4, zero_division='warn'))


# Desplique del modelo de la red neuronal

# Guardar el modelo

### Crear el directorio para guardar los modelos

In [18]:
# Crear el directorio de la aplicación
!mkdir -p modelos


In [ ]:

import joblib
# Obtener el state_dict (diccionario de pesos y sesgos)
state_dict = modelo_red_neuronal.state_dict()

# Guardar el state_dict
torch.save(state_dict, "./modelos/rna_polaridad.pth")
# Serializar el Label Encoder
joblib.dump(le, './modelos/le_polaridad.joblib')

  # Metadata
metadata = {
    'clase_modelo': 'RedNeuronal',
    'tam_entrada': tam_entrada,
    'tam_salida': tam_salida,
    'classes': le.classes_.tolist(),

}
joblib.dump(metadata, './modelos/metadata_polaridad.joblib')
    




#  Flask

 - ### Flask es un framework web: Proporciona las herramientas para crear aplicaciones web
 - ### Es un microframework: Sin dependencias a librerías externas
-  ### Framework ligero
-  #### [https://flask.palletsprojects.com](https://flask.palletsprojects.com)

# Instalar Flask

In [ ]:
!pip install flask

In [ ]:
import flask
print(flask.__version__)

# Crear el directorio de la aplicación Web

In [20]:
# Crear el directorio de la aplicación
!mkdir -p app_web


## Aplicacion Web en FLASK 

- ###  Una vez que cree la instancia app, se utiliza para gestionar las solicitudes web entrantes y enviar respuestas al usuario. 

- ###  @app.route es un decorador que convierte una función Python regular en una función vista de Flask

- ###  Convierte el valor de retorno de la función en una respuesta HTTP que se mostrará mediante un cliente HTTP

- ###  Pasa el valor '/' a @app.route() para indicar que esta función responderá a las solicitudes web para la URL /, que es la URL principal.

- ###  La función de vista inicio() devuelve la cadena ''¡Hola, mundo! Esta es mi primera aplicación Flask."​​ como respuesta.

## Crear el archivo inicio.py


In [ ]:
# Crear el archivo inicio.py

from flask import Flask

app = Flask(__name__)


@app.route('/')
def inicio():
    return '¡Hola, mundo! Esta es mi primera aplicación Flask.'

if __name__ == '__main__':
    app.run(debug=True)


### Ejecutar la aplicación


#### Desde la terminal, en el directorio de la aplicación

#### <span style="color:#0485CF"> python inicio.py </span>



```
* Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 947-720-893
127.0.0.1 - - [16/Feb/2024 14:56:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Feb/2024 14:56:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Feb/2024 14:58:34] "GET / HTTP/1.1" 200 -
```



#### Otra forma de ejecuta la aplicación web usando el puerto 5000
#### <span style="color:#0485CF"> flask --app inicio.py --debug run -p 5000 </span>



# Entrar al navegador para ver los resultados
http://127.0.0.1:5000


# Desplegar el clasificador en una interfaz WEB

### 1. Construir la aplicación WEB
- #### 1.1 Crear la página web (clasificador.html)
- #### 1.2 Crear punto de inicio de la aplicación web (inicio.py)
### 2. Copiar el modelo construido
### 3. Usar el modelo por medio de la aplicación web






# Ejemplo de página HTML

#### 1. Crear la carpeta "templates" en el directorio de la aplicación
#### 2. Tener una plantilla HTML para tu clasificador. Crear el archivo clasificador.html en la carpeta templates


# 1. Crear la página WEB
### 1.1 Crear el archivo: clasificador.html

In [ ]:
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Clasificador de Texto</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .container {
            text-align: center;
        }
        input[type="text"] {
            width: 500px;
            height: 100px;
            padding: 10px;
            margin-bottom: 10px;
            font-size: 16px;
        }
        button {
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <div class="container">
        <h1>Clasificador de Texto de Polaridad</h1>
        <form id="text-form" method="post" action="/clasificar">
            <input type="text" id="texto" name="texto" placeholder="Escribe tu texto para predecir" required>
            <br>
            <button type="submit">Clasificar</button>
        </form>

        <br>
        <br>
        <br> 
        {% if pred %}
        <h2 style="background-color:powderblue;">Resultado de la Clasificación</h2>
            <p style="background-color:plum;"><strong>Texto:</strong> {{texto}} </p>
            {% if pred == "positive" %}
                <p style="background-color:green;"><strong>Predicción:</strong> {{pred}} </p>
            {% elif pred == "negative" %}        
                <p style="background-color:tomato;"><strong>Predicción:</strong> {{pred}}  </p>
            {% elif pred == "neutral" %}        
                <p style="background-color:white;"><strong>Predicción:</strong> {{pred}} </p>
            {% endif %}
         {% endif %}
            
    </div>
</body>
</html>

### 1.2. En la carpeta de la aplicación Flask (app_web),  crear un archivo que contendrá la ruta del inicio de la página clasificador: inicio.py

### Archivo: inicio.py

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from ClasificadorRNA import ClasificadorRNA

app = Flask(__name__)



clasificador = ClasificadorRNA(fmetadata='./modelos/metadata_polaridad.joblib',
                                fmodelo='./modelos/rna_polaridad.pth',
                                flabelEnc='./modelos/le_polaridad.joblib',
                                fFastText='/Volumes/data/temp/MX.bin'
                                )


@app.route('/')
def index():
    return redirect(url_for('clasificar'))


@app.route('/clasificar', methods=['POST', 'GET'])
def clasificar():
    if request.method == 'POST':
        # Iniciar sesión exitosa, redirigir a otra página
        texto = request.form['texto']
        print(texto)
        prediccion = clasificador.predecir(texto)
        print(prediccion)
        return render_template('clasificador.html', texto=texto, pred=prediccion)
    else:
        return render_template('clasificador.html')


if __name__ == '__main__':
    app.run(debug=True)


### 1.3. En la carpeta de la aplicación Flask (app_web),  crear un archivo que contendrá el modelo de la red neuronal: red_neuronal.py

### Archivo: red_neuronal.py

In [ ]:
from torch import nn

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_salida):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        tam_entrada_capa_oculta_1 = 128
        tam_entrada_capa_oculta_2 = 8 
        self.fc1 = nn.Linear(tam_entrada, tam_entrada_capa_oculta_1)
        self.fc2 = nn.Linear(tam_entrada_capa_oculta_1, tam_entrada_capa_oculta_2)
        self.salida = nn.Linear(tam_entrada_capa_oculta_2, tam_salida)
        self.activacion1= nn.PReLU()
        self.activacion2= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.salida.weight)

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.activacion1(x)
        x = self.fc2(x)
        x = self.activacion2(x)
        x = self.salida(x)
        return x

### 1.4. En la carpeta de la aplicación Flask (app_web),  crear un archivo que contendrá el clasificador que invocará a la red neuronal: clasificador.py

### Archivo: clasificadorRNA.py

In [ ]:
from sklearn.preprocessing import LabelEncoder
import fasttext
from red_neruronal import RedNeuronal
import joblib
import torch

class ClasificadorRNA:
    
    def __init__(self, fmetadata, fmodelo, flabelEnc, fFastText):

        # TODO: Cargar word embeddings
        self.ft = fasttext.load_model(fFastText)
        # TODO: Cargar metadata primero (para verificar)
        self.metadata = joblib.load(fmetadata)
        # Cargar LabelEncoder
        self.label_encoder = joblib.load(flabelEnc)

        # Cargar modelo de red neuronal
        self.model = RedNeuronal(tam_entrada=self.metadata['tam_entrada'], 
                                tam_salida=self.metadata['tam_salida'])        
        # 4. Cargar los pesos de la red
        self.model.load_state_dict(torch.load(fmodelo))
    
    def _vectorizar(self, texto):
        print("Vectorizando texto")
        print(texto)
        return self.ft.get_sentence_vector(texto)

    def predecir(self, texto):
        vector_we = torch.tensor(self._vectorizar(texto), dtype=torch.float32)
        # Mete una dimension para hacer una matriz de 1 ejemplo y 300 características
        # [[feature1, feature2, ....]]
        vector_we = vector_we.unsqueeze(0)
        # Evalúa el modelo con el conjunto de validación
        self.model.eval()  # Establecer el modo del modelo a "evaluación"
        with torch.no_grad():  # No  calcular gradientes 
            y_pred = self.model(vector_we)
            print(f"pred model: {y_pred}")
            # Obtiene una única clase, la más probable
            y_pred = torch.argmax(y_pred, dim=1)
            pred = self.label_encoder.inverse_transform(y_pred)
            print(f"le pred: {pred}")
        return pred


# 2. Copiar el modelo del clasificador a la aplicación WEB

### 2.1. Copiar la carpeta "modelos" dentro del directorio de la aplicación WEB
### -  Archivo: rna_polaridad.pth
### -  Archivo: le_polaridad.joblib
### -  Archivo: metadata_polaridad.joblib


# 3. Ejecutar la aplicación WEB

#### <span style="color:#0485CF"> flask --app inicio.py --debug run -p 5000 </span>


## Pantalla del clasificador

<img src="figs/fig_app_web_clasificador.png" width="600">

# PROYECTO

# TODO: Crear modelos con pesado TF-IDF con los datos de agresividad
## Archivo de datos: data_aggressiveness_es.json

## Guías
### 1. Cargar los datos en un DataFrame
### 2. Inspeccionar los datos
#### 2. 1. Identificar las clases y el número de ejemplos de entrenamiento
- ####  El campo klass identifica la clase
    - #####  0: no agresivo
    - #####  1: no agresivo
#### 3. Construir la matriz de vectores
- ####  Preprocesamientos: minúsculas, normalizar texto, etc.
- ####  Aplicar Stemming
- ####  Crear un modelo con solo unigramas y evaluar su desempeño
- ####  Crear otro  modelo con unigramas y bigramas y evaluar su desempeño
- ####  Comparar los desempeños de los modelos 

#### 4. Dividir el conjunto de datos en datos de entrenamiento y prueba
#### 5. Construir el clasificador con el entrenamiento
#### 6. Evaluar el desempeño del clasificador
#### 7. Ejecutar el modelo de predicción desde una aplicación WEB


